In [1]:
import numpy as np
import sys
sys.path.append('../../package/utility/')
sys.path.append('../../package/robot_launcher/')
sys.path.append('../../package/kinematics_helper/')
from timer import Timer
from dynamixel_sdk import *
from transformation import *
from launch_robot import RobotLauncher


In [2]:
data = np.load('./result/18_허혜진.npz')
timestep = data['timestep']
qpos = data['qpos']
print(f"timestep shape: {timestep.shape}, qpos shape: {qpos.shape}")

timestep shape: (38069, 1), qpos shape: (38069, 5)


In [3]:
qpos

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  2.24000000e-01],
       [-3.18348518e-22, -5.32371949e-23,  3.76946359e-22,
        -3.65510975e-23,  2.24000000e-01],
       [-6.36697036e-22, -1.06474390e-22,  7.53892718e-22,
        -7.31021950e-23,  2.24000000e-01],
       ...,
       [ 3.98378039e-09, -1.54180780e-03,  6.42387037e-03,
        -8.93449745e-03,  2.24000000e-01],
       [ 3.98378039e-09, -1.54180780e-03,  6.42387037e-03,
        -8.93449745e-03,  2.24000000e-01],
       [ 3.98378039e-09, -1.54180780e-03,  6.42387037e-03,
        -8.93449745e-03,  2.24000000e-01]])

In [4]:
# velocity와 acceleration 계산
dt = np.diff(timestep, axis=0).flatten()  # (n-1,)
velocity = np.diff(qpos, axis=0) / dt[:, None]  # (n-1, 5)
acceleration = np.diff(velocity, axis=0) / dt[:-1, None]  # (n-2, 5)


# residual_torque_limits = np.array(
#     [4.07267981906041,
#     3.95198067018901,
#     3.97675265052986,
#     3.96028660022148,
#     3.9
#     ])
vel_limits = np.array(
    [4.78501013007516,
    4.64320014851508,
    4.67230486141645,
    4.6529588236653,
    4.6])
acc_limits = np.array(
    [231321.273714306,
    16418.1218822564,
    160356.382291855,
    23470.8936359572,
    20000.0])
# limit check
vel_exceed = np.abs(velocity) > vel_limits
acc_exceed = np.abs(acceleration) > acc_limits

# results
print("Velocity limit exceeded at indices (row, joint):", np.argwhere(vel_exceed))
print("Acceleration limit exceeded at indices (row, joint):", np.argwhere(acc_exceed))

Velocity limit exceeded at indices (row, joint): []
Acceleration limit exceeded at indices (row, joint): []


In [ ]:
if np.argwhere(vel_exceed).size == 0 and np.argwhere(acc_exceed).size == 0:
    tmr_cmd = Timer(_name='Command',_HZ=100,_MAX_SEC=np.inf,_VERBOSE=True)
    controller = RobotLauncher(name="omx")
    joint_names = ["joint1","joint2","joint3","joint4","gripper_crank_joint"]
    controller.set_specific_pid(pid=[(25,0,200)],id_list=[15])
    tmr_cmd.start()
    idx=0
    while idx<len(qpos):
        try:
            if tmr_cmd.do_run():
                controller.run(qpos[idx],joint_names)
                idx+=1
        except KeyboardInterrupt:
            break

Scan - baurdates: [1000000], id_range: range(0, 254)
[INFO] Scanning baudrate 1000000...
  - Found ID 11 (proto 2.0, model 1020)
  - Found ID 12 (proto 2.0, model 1020)
  - Found ID 13 (proto 2.0, model 1020)
  - Found ID 14 (proto 2.0, model 1020)
  - Found ID 15 (proto 2.0, model 1020)

[SUMMARY]
  Baud 1000000: [(11, 2.0, 1020), (12, 2.0, 1020), (13, 2.0, 1020), (14, 2.0, 1020), (15, 2.0, 1020)]
motor id list @ 1000000: [11, 12, 13, 14, 15]
Transitioning from current pose to zero pose over 2.0 seconds...
Current poses: ['0.001', '0.001', '0.001', '0.001', '0.223']
Control Hz: 50.0, Steps: 100, dt: 0.020s
Transition to zero pose completed!
[Command] start ([100]HZ. MAX_SEC:[infsec]).
[Command][990][9.9s] delayed! T:[10.0ms] But it took [16.8ms]. Acutally [0.0ms]
[Command][16252][162.5s] delayed! T:[10.0ms] But it took [15.9ms]. Acutally [0.0ms]
[Command][24327][243.3s] delayed! T:[10.0ms] But it took [15.1ms]. Acutally [0.0ms]
[Command][28267][282.7s] delayed! T:[10.0ms] But it took 